In [1]:
import xarray as xr
import numpy as np

ds = xr.open_dataset("data/tasminAdjust_FR-Metro_CMCC-CM2-SR5_ssp370_r1i1p1f1_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc",
                     engine="netcdf4")

# Boite l'Occitanie
lat_max = 45
lat_min = 42
lon_max = 3.5
lon_min = -1

lat = ds["lat"]
lon = ds["lon"]

mask = ((lat >= lat_min) & (lat <= lat_max) &
        (lon >= lon_min) & (lon <= lon_max))

# On prend pour l'instant juste la première date
tas = ds["tasminAdjust"].isel(time=0).where(mask)

On extrait les données dans un pandas dataframe

In [2]:
import geopandas as gpd
from shapely.geometry import Polygon

lat_b = ds["lat_bnds"].values
lon_b = ds["lon_bnds"].values

rows = []
geoms = []

ny, nx = tas.shape

for j in range(ny):
    for i in range(nx):
        if np.isnan(tas[j, i]):
            # on saute les pixels hors zone
            continue  
        
        lats = lat_b[j, i, :]
        lons = lon_b[j, i, :]

        poly = Polygon([(lons[k], lats[k]) for k in range(len(lats))])
        
        geoms.append(poly)
        rows.append({
            "tasminAdjust": float(tas[j, i])
        })

gdf = gpd.GeoDataFrame(rows, geometry=geoms, crs="EPSG:4326")

In [8]:
gdf.head()

,id,tasminAdjust,geometry
0,0,-2.038580,"POLYGON ((1.94955 42.31269, 2.04631 42.313, 2...."
1,1,-4.572150,"POLYGON ((2.04631 42.313, 2.14307 42.31322, 2...."
2,2,-2.137152,"POLYGON ((2.33659 42.3134, 2.43334 42.31336, 2..."
3,3,-1.953619,"POLYGON ((2.43334 42.31336, 2.5301 42.31322, 2..."
4,4,-0.884222,"POLYGON ((2.5301 42.31322, 2.62686 42.313, 2.6..."


réinitialiser l’index du GeoDataFrame

In [3]:
gdf = gdf.reset_index().rename(columns={"index": "id"})

conversion en degré

In [5]:
gdf["tasminAdjust"] = gdf["tasminAdjust"] - 273.15

cartographie avec folium

In [ ]:
import folium

# Centre de la carte
center = [
    gdf.geometry.centroid.y.mean(),
    gdf.geometry.centroid.x.mean()
]

m = folium.Map(location=center, zoom_start=7)

# Choropleth
folium.Choropleth(
    geo_data=gdf.to_json(),
    data=gdf,
    columns=["id", "tasminAdjust"],
    key_on="feature.properties.id",
    fill_color="RdYlBu_r",
    fill_opacity=0.7,
    line_opacity=0,
    nan_fill_color="white",
    legend_name="tasminAdjust (°C)"
).add_to(m)

# Ajout d'infobulles
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillOpacity": 0, 
        "weight": 0,
        "color": "transparent"
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["tasminAdjust"],
        aliases=["Température (°C) :"],
        localize=True
    )
).add_to(m)

m


/var/folders/_r/hyspm3rx34nf753l6xwvsrg00000gp/T/ipykernel_4198/3801722530.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.geometry.centroid.y.mean(),
/var/folders/_r/hyspm3rx34nf753l6xwvsrg00000gp/T/ipykernel_4198/3801722530.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.geometry.centroid.x.mean()
